# INVESTOR INTELLIGENCE AGENT - COMPANY INFORMATION RETRIEVAL

## TASK:

1. Retrieve Background of Company
2. Retrieve Stock Price of Company (Image?)
3. Retrieve Financial Metrics of Company
4. Time series analysis (Revenue, profit margins)


In [1]:
# Please uncomment and press shift+enter to install some requirements
# !pip install rpa 
# scikit-learn nltk lxml pystemmer

### Optionally, change the sourcecode of tagui command-line to launch the chrome in the incognito model to prevent some login issue

In [7]:
# set "incognito mode"
# !sed -i 's/chrome_started="$(uname -s)"; chrome_switches="--user-data-dir=chrome\/tagui_user_profile --remote-debugging-port=9222 about:blank"/chrome_started="$(uname -s)"; chrome_switches="--user-data-dir=chrome\/tagui_user_profile --remote-debugging-port=9222 -incognito about:blank"/g' ~/.tagui/src/tagui
# !sed -i "s/chrome_started=\"\$(uname -s)\"; chrome_switches=\"--user-data-dir=chrome\/tagui_user_profile --remote-debugging-port=9222 about:blank\"/chrome_started=\"\$(uname -s)\"; chrome_switches=\"--user-data-dir=chrome\/tagui_user_profile --remote-debugging-port=9222 --incognito about:blank\"/g" "C:/tagui/src/tagui"

# set "normal mode"
# !sed -i 's/chrome_started="$(uname -s)"; chrome_switches="--user-data-dir=chrome\/tagui_user_profile --remote-debugging-port=9222 -incognito about:blank"/chrome_started="$(uname -s)"; chrome_switches="--user-data-dir=chrome\/tagui_user_profile --remote-debugging-port=9222 about:blank"/g' ~/.tagui/src/tagui


---

In [19]:
# import rpa as t for the new vision of TagUI same function just different name
import tagui as t
import datetime

In [20]:
t.close()

[RPA][ERROR] - use init() before using close()


False

In [8]:
# Extract Company Information from Google Finance

def extract_company_info(company):
    try:
        t.init(visual_automation = True) # visual automation if keyboard automation required in subsequent code
        t.url('https://www.google.com/finance/?hl=en') # go to google finance website
        t.click('//c-wiz[2]//input[2]') # click on search bar and wait. Did not work when use type directly as first word to be searched is always missing. 
        t.wait(0.1) # introduced wait 0.1sec as per above comment to allow time before typing search
        t.type('//c-wiz[2]//input[2]',  company+'[enter]') # search for company. enter does not work somehow
        t.click('//*[@class="MkjOTb SAq8ff"][1]') # click the top return company. 

        Company_Full = t.read('//*[contains(@class,"zzDege")]') # read full name of company. act as confirmation to user's request as the search name used may not be complete
        live_price = t.read('//*[contains(@class,"YMlKec fxKbKc")]') # reads live stock exchange price
        Trade = t.read('//div[@class="PdOqHc"]') # reads stock market and ticker symbol used
        stock_market = Trade.split(" • ")[1] # extracts stock market name
        ticker_symbol = Trade.split(" • ")[0][slice(4,len(Trade.split(" • ")[0]))] # extracts ticker symbol used
        ceo = t.read('//span[@class="w4txWc oJeWuf"]/div[2]//a[@class="tBHE4e"]') # reads CEO of company
        about_company = t.read('//*[@class="bLLb2d"]')
        
        # financial_date
    
        return Company_Full, live_price, stock_market, ticker_symbol, ceo, about_company
    
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None, None, None, None, None, None

In [ ]:
# test
# company_info = extract_company_info(company) 
# print(company_info['live_price'])

In [6]:
# Snap and save Stock Price Chart
chart_duration = ["1day", "1year"] # Options of "1day", "5day", "1month", "6month", "ytd", "1year", "5year", and "max". Default set to 1 day and 1 year charts
for duration in chart_duration:
    t.click('//*[@id="' + duration + 'Tab"]') # click duration
    t.snap('//*[@class="ushogf"]',ticker_symbol + '/' + duration + '_chart_'+datetime.datetime.now().strftime("%Y%m%d_%H%M")+'.png') # snap  chart

# Working Code derived to click 1 year tab and snap  1 year chart before using array of chart_duration
# t.click('//*[@id="1yearTab"]') # click to display 1 year chart
# t.snap('//*[@class="ushogf"]',ticker_symbol+'/'+'1Y_chart_'+datetime.datetime.now().strftime("%Y%m%d_%H%M")+'.png') # snap 1 year chart

In [16]:
## Export to Excel as Stock Portfolio
import datetime
from openpyxl import Workbook, load_workbook

# Function to create or load workbook and add data
def update_excel(username, company_info):
    # Define file path based on username
    file_path = f"{username}_stock_profile.xlsx"

    try:
        # Load existing workbook
        workbook = load_workbook(file_path)
        worksheet = workbook.active
    except FileNotFoundError:
        # If workbook doesn't exist, create a new one
        workbook = Workbook()
        worksheet = workbook.active
        # Add headers to the first row
        worksheet.append(["Date", "Company", "Live Price", "Stock Market", "Ticker Symbol", "CEO", "Company Description"])

    # Append data to the workbook
    row_data = [datetime.datetime.now()] + list(company_info)
    worksheet.append(row_data)

    # Save the workbook
    workbook.save(file_path)

In [18]:
# Example usage:
username = "ky"  # Define username
company = 'apple' # User input company of interest

company_info = extract_company_info(company) # stores extracted information
if company_info[0]:  # Check if data is not None
    Company_Full, live_price, stock_market, ticker_symbol, ceo, about_company = company_info
    # Process retrieved data
else:
    print("Company information retrieval failed.")

update_excel(username, company_info)

## Summary
Great recap! Let's now deep drive in the this process
- 1. we upload a image
- 2. Bing get the image file and do some magic process, then back the result
- 3. we download top threes similar images

Intelligent process automation process
- We use tagui-bot to upload and download the image;
- We use tagui-bot to retrieve similar image from Bing Image Search;


<div class="alert alert-info alertdanger" style="margin-top: 20px">
    <h1>
       Virtual Assistant
    </h1>
    <p>
       for Job Applicatio & Cover Letter
    </p>
</div>

## IPA Task

We are to build a JobApplicationAssistant which will monitor "Job Opportunities" email from the provided email. Based on the user's "Resume/CV" and job desciption, the agent can generate tailored cover letters for job appication via eamil.

## IPA Solution

### 1. First: Monitor and retieve job opportunities 

    Get the job description from the target email account
        - Login the mail
        - Search the job description emails
        - Download the job description files
        
### 2. Second: Write cover letter 

    Use local AI module to conduct text/language processing and generate the coverletters based on the Job Description and the Resume

### 3. Third: Send job application via email 

    In this part, we use both pure python method and tagui to send the email to the hiring company


<div class="alert alert-success alertdanger" style="margin-top: 20px">
    <h1>
        First: Monitor and retieve job opportunities
    </h1>
    <p>
        Get the job description from the target email account
    </p>
</div>

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    <h1>
         Workshop Quiz:
    </h1>
    <p>
         Login the target Email Account
    </p>
    <p>
         <b>(Recap earlier IPA workshop Tutorial)</b>
    </p>
        <p>
        <b>Check TagUI-Python functions on TagUI-Python(<a href="https://github.com/tebelorg/RPA-Python">https://github.com/tebelorg/RPA-Python</a>) </b>
    </p>
</div>

In [4]:
import tagui as t
import os

CURRENT_PATH = os.getcwd()

print(CURRENT_PATH)

C:\Users\kenny\Desktop\S-IPA-Workshop-master\workshop2\Virtual-Assistant-Job


In [5]:
def hover_and_read(selector):
    t.hover(selector)
    str = t.read(selector)
    return str

In [6]:
t.close()

[RPA][ERROR] - use init() before using close()


False

In [7]:
import json

email_info = None
with open("./email_info.json",'r') as load_f: # Ask lecturer for this ./email_info.json file
    email_info = json.load(load_f)
    
email_info = email_info['1']
# email_info

In [51]:
# name = "XU JIACHEN"
# phone = "+65 93752168"
name = "Virtual Assistant of Mr. KY"
phone = "+65 6601 3161"

# Please uncomment next line please change to your own email address
# company_emails = ['liyingxujiachen@gmail.com']
company_emails = ['kysem2@hotmail.com']
resume_file_path = os.path.join(CURRENT_PATH,"resume/resume.pdf")
# maybe you chang to your own resume

In [18]:
# Write your code from here:


---

In [19]:
# Or use below 'filling the blank' coding template:

In [37]:
# Write your code below
def loginEmail_outlook(email_account,email_pwd):
    try:
        t.init(visual_automation = True)
        t.url('https://login.live.com/login.srf?wa=wsignin1.0&rpsnv=13&ct=1580788659&rver=7.0.6737.0&wp=MBI_SSL&wreply=https%3a%2f%2foutlook.live.com%2fowa%2f%3fnlp%3d1%26RpsCsrfState%3dd234420e-f55a-d62c-a8e6-c1c9a31e4e54&id=292841&aadredir=1&CBCXT=out&lw=1&fl=dob%2cflname%2cwld&cobrandid=90015')
        t.type('//*[@id="i0116"]',  email_account + '[enter]')
        t.wait(3)
#         t.click('//*[@id="idSIButton9"]')  # Click the Next button
#         t.wait(0.5)
        t.type('//*[@id="i0118"]', email_pwd + '[enter]')
#         t.wait(0.5)
#         t.click('//*[@id="idSIButton9"]')  # Click the Sign in button
        t.wait(3)
        if (t.present('//input[@id="idBtn_Back"]')):
            t.click('//input[@id="idBtn_Back"]')
        t.wait(5)
    except:
        t.close()

In [27]:
try:
    loginEmail_outlook(email_info['email'],email_info['password'])
finally:
    t.close()

In [22]:
# Uncommnet below, then press Shift+Enter for solution
# codehelper.hint_for_ipa_recap_two()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    <h1>
         Workshop Quiz:
    </h1>
    <p>
         Download the Job Description files from email being monitored(After login)
    </p>
            <p>
        <b>Check TagUI-Python functions on TagUI-Python(<a href="https://github.com/tebelorg/RPA-Python">https://github.com/tebelorg/RPA-Python</a>) </b>
    </p>
</div>

In [43]:
# Write your code from here:


---

In [24]:
# Following lines are step-by-step codes used during testing prior to completing the blank coding template

In [16]:
# loginEmail_outlook(email_info['email'],email_info['password'])
# t.wait(1)

# t.click('//*[@id="topSearchInput"]')
# t.type('//*[@id="topSearchInput"]','from:kenny chua job opportunity')
# t.click('//button[@aria-label="Search"]')
# t.wait(3)
num_email=t.count('//div[@class="XG5Jd JtO0E"]')
print('The number of job opportunity emails is: ',num_email)

The number of job opportunity emails is:  8


In [24]:
for n in range(1,num_email+1):
    t.click(f'(//div[@data-convid])[{n}]')
    t.wait(1)


In [33]:
jd_files=[]
title = 'hello'
t.click(f'(//div[@data-convid])[{1}]')
if t.present('//div[@class="VlyYV PQeLQ QEiYT"]'):
    # jd_files.append(os.path.join(CURRENT_PATH,hover_and_read('//div[@class="VlyYV PQeLQ QEiYT"]').split("location=./")[-1]))
    title = hover_and_read('//div[@class="VlyYV PQeLQ QEiYT"]/@title')
print(title)


IS05 FT Student Timetable (AY2023_2024) V6.0.pdf


In [36]:
jd_files.append(os.path.join(CURRENT_PATH,hover_and_read('//div[@class="VlyYV PQeLQ QEiYT"]/@title')))
print("Those job opportunities files ",jd_files)


Those job opportunities files  ['C:\\Users\\kenny\\Desktop\\S-IPA-Workshop-master\\workshop2\\Virtual-Assistant-Job\\IS05 FT Student Timetable (AY2023_2024) V6.0.pdf']


In [11]:
t.close()

True

In [78]:
# Or use below 'filling the blank' coding template:

In [38]:
# maybe you will surprise about some xpath using unstable class name or other attributes
def getJobDescription():
    try:
        
        t.click('//*[@id="topSearchInput"]')
        t.type('//*[@id="topSearchInput"]','from:kenny chua job opportunity')
        t.click('//button[@aria-label="Search"]')
        t.wait(3)
        num_email=t.count('//div[@class="XG5Jd JtO0E"]')
        print('The number of job opportunity emails is: ',num_email)
        jd_files=[]
        for n in range(1,num_email+1):
            t.click(f'(//div[@data-convid])[{n}]')
            t.wait(1)
            if t.present('//div[@class="VlyYV PQeLQ QEiYT"]'):
                jd_files.append(os.path.join(CURRENT_PATH,hover_and_read('//div[@class="VlyYV PQeLQ QEiYT"]/@title')))
                print("Those job opportunities files ",jd_files)
#         for n in range(1,num_email+1):
#             t.click(f'(//div[@data-convid"])[{n}]')
#             jd_files.append(os.path.join(CURRENT_PATH,hover_and_read('//div[@class="JA9Uf YyULm"]').split("location=./")[-1]))
#             print("Thoes job opportunities files ",jd_files)
        return jd_files
    finally:
        t.close()

In [39]:
t.close()

True

In [40]:
try:
    loginEmail_outlook(email_info['email'],email_info['password'])
    t.wait(15)
    jd_files = list(set(getJobDescription()))
finally:
    t.close()

The number of job opportunity emails is:  8
Those job opportunities files  ['C:\\Users\\kenny\\Desktop\\S-IPA-Workshop-master\\workshop2\\Virtual-Assistant-Job\\20230908A_Asthma Exacerbation Predictive_SingHealth_(EBAC)(IS).pdf']
Those job opportunities files  ['C:\\Users\\kenny\\Desktop\\S-IPA-Workshop-master\\workshop2\\Virtual-Assistant-Job\\20230908A_Asthma Exacerbation Predictive_SingHealth_(EBAC)(IS).pdf', 'C:\\Users\\kenny\\Desktop\\S-IPA-Workshop-master\\workshop2\\Virtual-Assistant-Job\\IS05 FT Student Timetable (AY2023_2024) V6.0.pdf']
Those job opportunities files  ['C:\\Users\\kenny\\Desktop\\S-IPA-Workshop-master\\workshop2\\Virtual-Assistant-Job\\20230908A_Asthma Exacerbation Predictive_SingHealth_(EBAC)(IS).pdf', 'C:\\Users\\kenny\\Desktop\\S-IPA-Workshop-master\\workshop2\\Virtual-Assistant-Job\\IS05 FT Student Timetable (AY2023_2024) V6.0.pdf', 'C:\\Users\\kenny\\Desktop\\S-IPA-Workshop-master\\workshop2\\Virtual-Assistant-Job\\20230908A_Asthma Exacerbation Predictive_

In [41]:
jd_files

['C:\\Users\\kenny\\Desktop\\S-IPA-Workshop-master\\workshop2\\Virtual-Assistant-Job\\20230908A_Asthma Exacerbation Predictive_SingHealth_(EBAC)(IS).pdf',
 'C:\\Users\\kenny\\Desktop\\S-IPA-Workshop-master\\workshop2\\Virtual-Assistant-Job\\IS05 FT Student Timetable (AY2023_2024) V6.0.pdf']

In [42]:
# Uncommnet below, then press Shift+Enter for solution
# codehelper.hint_for_ipa_exercise_one()

In [43]:
t.close()

[RPA][ERROR] - use init() before using close()


False

<div class="alert alert-success alertdanger" style="margin-top: 20px">
    <h1>
        Second: Write cover letter
    </h1>
    <p>
        Use local AI module to conduct text/language processing and generate the coverletters based on the Job Description and the Resume
    </p>
</div>

In [49]:
resume_file_path

'C:\\Users\\kenny\\Desktop\\S-IPA-Workshop-master\\workshop2\\Virtual-Assistant-Job\\resume/resume.pdf'

In [64]:
from ipa import CoverLetterGenerator

os.chdir(CURRENT_PATH)

gen = CoverLetterGenerator(resume_file_path,jd_files,name,phone,email_info['email'])

import time
starttime = time.clock()
cover_letters = gen.generate()
endtime = time.clock()
print('Running time: %s Mins'%((endtime-starttime)/60))

ModuleNotFoundError: No module named 'Stemmer'

In [44]:
# if the result is empty, that means your resume doesn't match those job postions, don't worry.
# You can use your own resume
cover_letters

['/media/sf_vm_shared_folder/git/S-IPA-Workshop/workshop2/Virtual-Assistant-Job/ipa/coverletter_20200309/templates_0.pdf',
 '/media/sf_vm_shared_folder/git/S-IPA-Workshop/workshop2/Virtual-Assistant-Job/ipa/coverletter_20200309/templates_1.pdf',
 '/media/sf_vm_shared_folder/git/S-IPA-Workshop/workshop2/Virtual-Assistant-Job/ipa/coverletter_20200309/templates_2.pdf',
 '/media/sf_vm_shared_folder/git/S-IPA-Workshop/workshop2/Virtual-Assistant-Job/ipa/coverletter_20200309/templates_3.pdf',
 '/media/sf_vm_shared_folder/git/S-IPA-Workshop/workshop2/Virtual-Assistant-Job/ipa/coverletter_20200309/templates_4.pdf',
 '/media/sf_vm_shared_folder/git/S-IPA-Workshop/workshop2/Virtual-Assistant-Job/ipa/coverletter_20200309/templates_5.pdf']

In [45]:
import shutil

for index,cletter in enumerate(cover_letters):
    shutil.copy(cletter,
            './coverletter/coverleter_'+str(index)+".pdf")

<div class="alert alert-success alertdanger" style="margin-top: 20px">
    <h1>
        Third: Send back application Email
    </h1>
    <p>
        In this part, we use both pure python method and tagui to send the email to the hiring company
    </p>
</div>

**Send email by Python:**

In [35]:
import smtplib
import imghdr
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import random

imgType_list = {'jpg','bmp','png','jpeg','rgb','tif'}

# def send(sender_account,sender_password,toaddrs,email_text):
#     try:
#         server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
#         server.ehlo()
#         server.login(sender_account, sender_password)
#         server.sendmail(sender_account, toaddrs, email_text)
#         server.close()
#         print('Email sent!')
#     except:
#         print('Something went wrong...') 

def send(sender_account,sender_password,toaddrs,email_text):
    try:
        server = smtplib.SMTP('smtp.office365.com', 587)
        server.starttls()
        server.login(sender_account, sender_password)
        print ('server working fine')
        server.sendmail(sender_account, toaddrs, email_text)
        server.close()
        print('Email sent!')
    except Exception as e:
        print(e)
        print('Something went wrong...') 

def sendmail(sender_account,sender_password,toaddrs,subject,content,attachments):

    m = MIMEMultipart()
    
    textApart = MIMEText(content)
    
    m.attach(textApart)
    
    
    for attach in attachments:
        if imghdr.what(attach) in imgType_list:
            imageApart = MIMEImage(open(attach, 'rb').read(), attach.split('.')[-1])
            imageApart.add_header('Content-Disposition', 'attachment', filename=attach)
            m.attach(imageApart)
        else:
            docApart = MIMEApplication(open(attach, 'rb').read())
            docApart.add_header('Content-Disposition', 'attachment', filename=attach)
            m.attach(docApart)
            
    m['Subject'] = subject
    
    send(sender_account,sender_password,toaddrs,m.as_string()) 

In [36]:
company_emails

['bwbw75@gmail.com']

In [56]:
for index,_ in enumerate(cover_letters):
    sent_from = email_info['email']
    to = [random.choices(company_emails)]
    subject = 'Job application - cover letter & resume (Python)'
    body = "Dear Sir/Madam, \n \
    I am eager to apply for your position. And consider myself a best fit. \n \
    (insert many nice words of your choice) \n\n \
    Warm regards, \n" + name
    attachments = ['./coverletter/coverleter_'+str(index)+".pdf",'./resume/resume.pdf']
    sendmail(email_info['email'],email_info['password'],to,subject,body,attachments)

server working fine
Email sent!
server working fine
Email sent!


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
    <h1>
         Workshop Quiz:
    </h1>
    <p>
         Send email by TagUI
    </p>    
    <p>
         Recall "CSS Selector Exericse in earlier workshop --- use the Bing Image Search to search the related image" about How to use TagUI upload file
    </p>
                <p>
        <b>Check TagUI-Python functions on TagUI-Python(<a href="https://github.com/tebelorg/RPA-Python">https://github.com/tebelorg/RPA-Python</a>) </b>
    </p>
</div>

In [ ]:
# Write your code from here:
# Following lines are step-by-step codes used during testing prior to completing the blank coding template

In [71]:
loginEmail_outlook(email_info['email'],email_info['password'])


In [72]:
t.click('//*[@id="id__132"]') # the xpath for the new email button
t.wait(2)


True

In [48]:
import random

In [50]:
to = random.choices(company_emails)
subject = 'Job application - cover letter & resume (TagUI)'

t.type('//*[@id="docking_InitVisiblePart_0"]/div/div[3]/div[1]/div/div[3]/div/div/div[1]'," ".join(to))
t.wait(2)
# t.click('//*[@id="TextField521"]')
# t.type('//*[@id="TextField521"]',subject)
t.click("//input[contains(@class,'ms-TextField-field') and contains(@aria-label, 'subject')]")
t.type("//input[contains(@class,'ms-TextField-field') and contains(@aria-label, 'subject')]",subject)
t.wait(2)

True

In [52]:
body = "Dear Sir/Madam, \n \
    I am eager to apply for your position. And consider myself a best fit. \n \
    (insert many nice words of your choice) \n\n \
    Warm regards, \n" + name

t.click('//*[@id="editorParent_1"]/div')
t.type('//*[@id="editorParent_1"]/div',body)


True

In [56]:
attachments = ['./coverletter/coverleter_'+str('1')+".pdf",'./resume/resume.pdf']
for file in attachments:
    t.upload("div>input[type='file']+input",file)# the css selector of upload files

In [57]:
t.click('//*[@id="docking_InitVisiblePart_0"]/div/div[2]/div[1]/div/span/button[1]')


True

In [75]:
t.close()

True

---

In [ ]:
# Or use below 'filling the blank' coding template:

In [74]:
def sendemail_tagui_outlook(email_account,email_pwd,to,subject,body,attachments):
    try:
        loginEmail_outlook(email_account,email_pwd)
        t.wait(5)

        t.click('//*[@id="id__132"]') # the xpath for the new email button
        t.wait(2)
        t.type('//*[@id="docking_InitVisiblePart_0"]/div/div[3]/div[1]/div/div[3]/div/div/div[1]'," ".join(to)) # type send to
        t.wait(2)
#         t.click('//*[@id="TextField521"]') # click subject field
#         t.type('//*[@id="TextField521"]',subject)
        t.click("//input[contains(@class,'ms-TextField-field') and contains(@aria-label, 'subject')]")
        t.type("//input[contains(@class,'ms-TextField-field') and contains(@aria-label, 'subject')]",subject)
        t.wait(2)
        t.click('//*[@id="editorParent_1"]/div') # click content field
        t.type('//*[@id="editorParent_1"]/div',body)
        
        for file in attachments:
            t.upload("div>input[type='file']+input",file)# the css selector of upload files
            
        t.click('//*[@id="docking_InitVisiblePart_0"]/div/div[2]/div[1]/div/span/button[1]') #click send button
        
        t.wait(10)
    finally:
        t.close()

In [62]:
cover_letters = [r'C:\Users\kenny\Desktop\S-IPA-Workshop-master\workshop2\Virtual-Assistant-Job\coverletter\coverleter_0.pdf']
#                 r'C:\Users\kenny\Desktop\S-IPA-Workshop-master\workshop2\Virtual-Assistant-Job\coverletter\coverleter_1.pdf',
#                 r'C:\Users\kenny\Desktop\S-IPA-Workshop-master\workshop2\Virtual-Assistant-Job\coverletter\coverleter_2.pdf',
#                 r'C:\Users\kenny\Desktop\S-IPA-Workshop-master\workshop2\Virtual-Assistant-Job\coverletter\coverleter_3.pdf',
#                 r'C:\Users\kenny\Desktop\S-IPA-Workshop-master\workshop2\Virtual-Assistant-Job\coverletter\coverleter_4.pdf',
#                 r'C:\Users\kenny\Desktop\S-IPA-Workshop-master\workshop2\Virtual-Assistant-Job\coverletter\coverleter_5.pdf']

In [76]:
for index,_ in enumerate(cover_letters):
    sent_from = email_info['email']
    to = random.choices(company_emails)
    subject = 'Job application - cover letter & resume (TagUI)'
    body = "Dear Sir/Madam, \n \
    I am eager to apply for your position. And consider myself a best fit. \n \
    (insert many nice words of your choice) \n\n \
    Warm regards, \n" + name
    attachments = ['./coverletter/coverleter_'+str(index)+".pdf",'./resume/resume.pdf']
    sendemail_tagui_outlook(email_info['email'],email_info['password'],to,subject,body,attachments) 

In [77]:
# Uncommnet below, then press Shift+Enter for solution
# codehelper.hint_for_ipa_exercise_two()

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <a href="https://github.com/isa-ipa">
        <img src="" width="60" align="right">
     </a>
     <h1>
         End of Workshop
     </h1>
</div>

---